# Rasterize Natural Gas Pipeline Network
Creates a raster version at the same extent and cell size of the MIT cost surface.

In [ ]:
import rasterio
from rasterio import features
import geopandas as gpd
import pandas as pd
import numpy as np

#### Import the full MIT cost surface raster
This is used to extract the extent and cell size of the created pipeline raster

In [ ]:
#Load the cost surface raster
cost_surface_dataset = rasterio.open('../data/MIT_Surface_Full/costsurface500m.img')
#Get the surface 
cost_surface = cost_surface_dataset.read(1)
#Extract the coordinate reference system
cost_crs = cost_surface_dataset.crs
#Get the metadata
cost_meta = cost_surface_dataset.meta

#### Import the REXTAG natural gas feature class
*NOTE: This is proprietary dataset stored on the secure server*

In [ ]:
#Read in features
pipelines_gdb = 'C:\\Workspace\\Rextag_7May2020\\Rextag_Zone2_Natural_Gas.gdb'
pipelines_lyr = 'NaturalGasPipelines'

In [ ]:
#Read in pipeline dataset as a geodataframe
gdf = gpd.read_file(pipelines_gdb,driver='FileGDB',layer=pipelines_lyr)

#### Preprocess the dataset
* Extract only pipelines tagged as "Operational"
* Reduce records to only VA, NC, and SC
* Transform the selected records to match the coordinate system of the MIT raster

In [ ]:
#Create masks
operational_mask = gdf['STATUS'] == 'Operational'
state_mask = gdf['STATE_NAME'].isin(['Virginia','North Carolina','South Carolina'])

In [ ]:
#Apply masks
gdf_operational = gdf.loc[operational_mask & state_mask].reset_index()

In [ ]:
#Transform to same crs as cost surface
gdf_operational = gdf_operational.to_crs(cost_crs)

#### Map pipeline "TYPE" values to numeric codes

In [ ]:
#Show a list of unique values
gdf_operational.TYPE.unique()

In [ ]:
#Create a dictionary to map values
typeDict = {'Transmission':1,
            'Distribution':2,
            'Gathering':3}
#Map the values to the dataframe
gdf_operational['TYPE_CODE'] = gdf_operational['TYPE'].apply(lambda x: typeDict[x])

In [ ]:
#Save map as dataframe and export as csv
dfMap = pd.DataFrame((typeDict.keys(),typeDict.values())).T
dfMap.columns=("TYPE","TYPE_CODE")
dfMap.to_csv('..\\data\\processed\\USA_pipes_operational_TYPE_LUT.csv',index=False)

#### Rastersize features using TYPE_CODE as the output cell value

In [ ]:
#Create list of shape/value tuples to rasterize
out_shapes = zip(gdf_operational.geometry,gdf_operational['TYPE_CODE'])

In [ ]:
#Convert features to raster
image = features.rasterize(shapes=out_shapes,
                           out_shape=cost_surface_dataset.shape,
                           transform=cost_surface_dataset.transform,
                           all_touched=False)

#### Explort the result to a GeoTIFF

In [ ]:
#Get the minimum dtype needed for values
min_dtype = rasterio.dtypes.get_minimum_dtype(gdf_operational.TYPE_CODE)
min_dtype

In [ ]:
#Update metadata for output
cost_meta = cost_surface_dataset.meta
cost_meta.update(
    {'driver':'GTiff',
     'nodata':255,
     'dtype':'uint8'
    })

In [ ]:
#Export as tiff
with rasterio.open('..\\data\\processed\\USA_pipes_operational.tif','w',**cost_meta) as dst:
    dst.write(image, indexes=1)